In [10]:
import pandas as pd

In [15]:
t = pd.read_csv('datasets/rossmann-store-sales/train.csv')

/tmp/ipykernel_76/1197880277.py:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  t = pd.read_csv('datasets/rossmann-store-sales/train.csv')


In [16]:
t = t.tail(500)

In [17]:
t = t.sort_values('Date', ascending=True).reset_index(drop=True)

In [28]:
t.head(20)

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,615,2,2013-01-01,0,0,0,0,a,1
1,956,2,2013-01-01,0,0,0,0,a,1
2,955,2,2013-01-01,0,0,0,0,a,1
3,954,2,2013-01-01,0,0,0,0,a,1
4,953,2,2013-01-01,0,0,0,0,a,1
5,952,2,2013-01-01,0,0,0,0,a,1
6,951,2,2013-01-01,0,0,0,0,a,1
7,950,2,2013-01-01,0,0,0,0,a,1
8,949,2,2013-01-01,0,0,0,0,a,1
9,948,2,2013-01-01,4491,1039,1,0,a,1


---

In [1]:
import os
import json
from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    IntegerType,
    DateType
)
from pyspark.sql.functions import from_json, col
from db_utils import prepare_db

POSTGRES_PORT = os.getenv('POSTGRES_PORT', '5432')
POSTGRES_CONNECTION_URL = os.getenv('POSTGRES_CONNECTION_URL', f'jdbc:postgresql://postgres:{POSTGRES_PORT}/spark_pg_db')
KAFKA_TOPIC = os.getenv('KAFKA_TOPIC', 'sale_rossman_store')

In [2]:
prepare_db()

Preparing database tables...
Done database preparation


In [3]:
spark = (
        SparkSession.builder.appName("PySpark to Postgres")
        .config(
            "spark.jars.packages",
            "org.postgresql:postgresql:42.5.4,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"
        )
        .getOrCreate()
    )

:: loading settings :: url = jar:file:/opt/anaconda3/envs/computer-viz-dl/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ariya/.ivy2/cache
The jars for the packages stored in: /home/ariya/.ivy2/jars
org.postgresql#postgresql added as a dependency
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3e4e1d00-33d2-498e-a6fb-a5b0c4d0cd90;1.0
	confs: [default]
	found org.postgresql#postgresql;42.5.4 in central
	found org.checkerframework#checker-qual;3.5.0 in central
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.cod

In [4]:
# create initial df
df = (
            spark.readStream.format("kafka")
            .option("kafka.bootstrap.servers", "kafka:9092")
            .option("subscribe", KAFKA_TOPIC)
            .option("startingOffsets", "earliest")
            .load()
        )

In [5]:
DB_FIELDS = ['store', 'dayofweek', 'date', 'sales', 'customers', 'open', 'promo',
       'stateholiday', 'schoolholiday']

In [6]:
# create final df
schema = StructType(
        [
            StructField('store', IntegerType(), True),
            StructField('dayofweek', IntegerType(), True),
            StructField('date', DateType(), True),
            StructField('sales', IntegerType(), True),
            StructField('customers', IntegerType(), True),
            StructField('open', IntegerType(), True),
            StructField('promo', IntegerType(), True),
            StructField('stateholiday', StringType(), True),
            StructField('schoolholiday', StringType(), True),
        ]
    )
df_out = (
    df.selectExpr("CAST(value AS STRING)")
    .select(from_json(col("value"), schema).alias("data"))
    .select("data.*")
)

In [7]:
df_out

DataFrame[store: int, dayofweek: int, date: date, sales: int, customers: int, open: int, promo: int, stateholiday: string, schoolholiday: string]

In [8]:
POSTGRES_PROPERTIES = {
    "user": "spark_user",
    "password": "SuperSecurePwdHere",
    "driver": "org.postgresql.Driver",
}

In [10]:
# start_streaming
# Read existing data from PostgreSQL

print("Start streaming ...")
query = (df_out.writeStream.foreachBatch(
    lambda batch_df, batch_id: (
        batch_df.write.jdbc(
            POSTGRES_CONNECTION_URL, "rossman_sales", "append", properties=POSTGRES_PROPERTIES
        )
    )
).trigger(once=True)
.option("checkpointLocation", "./spark_streaming_checkpoints") # set checkpoints so pyspark won't reread the same messages
.start()
)
query.awaitTermination()

Start streaming ...


24/03/17 07:04:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/03/17 07:04:13 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
